# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [1]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [2]:
import pandas as pd
import numpy as np
import re
import os
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

from IPython.display import display, Markdown

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [3]:
# sys.path

In [4]:
from isadream.models import utils
from isadream import io
from isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [5]:
# A demo json file is provided.
from isadream import config, demos, demo_path

---

In [6]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR"]))
node = io.parse_node_json(demo_json)

In [7]:
node.assays[1].factors

[]

In [8]:
node.as_markdown

"# Sipos 2006, Talanta NMR Figures\n\n**Description**: Extracted figures.\n\n**Submission Date**: *2018-5-25*\n\n**Public Release Date**: *2006-03-13*\n\n\n---\n## Assays\n### Potassium Hydroxide Assay\n#### Potassium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**K+** stoichiometry: 1.0\n\n**OH-** stoichiometry: 1.0\n\n**Comment00013**: I manually pulled this points out with a web tool.\n\n### Lithium Hydroxide Assay\n#### Lithium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**OH-** stoichiometry: 1.0\n\n**Li+** stoichiometry: 1.0\n\n### Sodium Hydroxide Assay\n#### Sodium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**OH-** stoichiometry: 1.0\n\n**Na+** stoichiometry: 1.0\n\n## Samples\n#### Aluminate Solution\n**('Material Property', 'Purity by Weight', 'Percent')**: 0.98\n\n**Al(III)** stoichiometry: 1.0\n\n**('Measurement Condition', '', 'Molar')**: 0.005\n\n**('Material Property', 'Purity by Weight', 'Percent')**: 0.98\n\n**Al(II

In [10]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR"]))
node = io.parse_node_json(demo_json)

x_groups=(('Total Aluminate Concentration', ('Molar',), ("Al",)),
          ('Counter Ion Concentration', ('Molar',),
           ("Na+", "Li+", "Cs+", "K+")),
          ('Counter Ion', ('Species',), ("Na+", "Li+", "Cs+", "K+",)),
          ('Base Concentration', ('Molar',), ("OH-",)))

y_groups=(('27 Al ppm', ('ppm',), ("Al",)),)

test_groups = x_groups + y_groups

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
# node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,Total Aluminate Concentration,sample_node,Counter Ion Concentration,Counter Ion,Base Concentration,27 Al ppm
0,42521992-c7e3-4711-8625-ea11310378ea,198457bc-bfc5-40a5-97ba-e2cd7582b4ba,0.005,a716fecc-2562-47de-a750-6b1e548c83ae,2.93,K+,2.93,79.90
1,42521992-c7e3-4711-8625-ea11310378ea,198457bc-bfc5-40a5-97ba-e2cd7582b4ba,0.005,a716fecc-2562-47de-a750-6b1e548c83ae,4.92,K+,4.92,79.84
2,42521992-c7e3-4711-8625-ea11310378ea,198457bc-bfc5-40a5-97ba-e2cd7582b4ba,0.005,a716fecc-2562-47de-a750-6b1e548c83ae,6.85,K+,6.85,79.72
3,42521992-c7e3-4711-8625-ea11310378ea,198457bc-bfc5-40a5-97ba-e2cd7582b4ba,0.005,a716fecc-2562-47de-a750-6b1e548c83ae,9.13,K+,9.13,79.66
4,42521992-c7e3-4711-8625-ea11310378ea,198457bc-bfc5-40a5-97ba-e2cd7582b4ba,0.005,a716fecc-2562-47de-a750-6b1e548c83ae,10.71,K+,10.71,79.66
0,1c89cf78-0487-409f-9598-4a23acd8f8dc,eeb1c5d6-ed39-41cc-919b-bab39808c599,0.005,3e06b5ba-76a0-451a-bc59-3a3629eccb8f,0.66,Li+,0.66,79.92
1,1c89cf78-0487-409f-9598-4a23acd8f8dc,eeb1c5d6-ed39-41cc-919b-bab39808c599,0.005,3e06b5ba-76a0-451a-bc59-3a3629eccb8f,1.10,Li+,1.10,79.84
2,1c89cf78-0487-409f-9598-4a23acd8f8dc,eeb1c5d6-ed39-41cc-919b-bab39808c599,0.005,3e06b5ba-76a0-451a-bc59-3a3629eccb8f,1.64,Li+,1.64,79.66
3,1c89cf78-0487-409f-9598-4a23acd8f8dc,eeb1c5d6-ed39-41cc-919b-bab39808c599,0.005,3e06b5ba-76a0-451a-bc59-3a3629eccb8f,2.14,Li+,2.14,79.54
4,1c89cf78-0487-409f-9598-4a23acd8f8dc,eeb1c5d6-ed39-41cc-919b-bab39808c599,0.005,3e06b5ba-76a0-451a-bc59-3a3629eccb8f,2.59,Li+,2.59,79.36


In [10]:
help(bk.transform.factor_cmap)

Help on function factor_cmap in module bokeh.transform:

factor_cmap(field_name, palette, factors, start=0, end=None, nan_color='gray')
    Create a ``DataSpec`` dict to apply a client-side
    ``CategoricalColorMapper`` transformation to a ``ColumnDataSource``
    column.
    
    Args:
        field_name (str) : a field name to configure ``DataSpec`` with
    
        palette (seq[color]) : a list of colors to use for colormapping
    
        factors (seq) : a sequences of categorical factors corresponding to
            the palette
    
        start (int, optional) : a start slice index to apply when the column
            data has factors with multiple levels. (default: 0)
    
        end (int, optional) : an end slice index to apply when the column
            data has factors with multiple levels. (default: None)
    
        nan_color (color, optional) : a default color to use when mapping data
            from a column does not succeed (default: "gray")
    
    Returns:
   

In [32]:
metadata = [md for _, md in node_assays]
metadata
# md = collections.ChainMap(*metadata)
# md

[{'11110c3a-c816-43e2-9a75-9bae1db010e3': ({'node_title': 'Sipos 2006, Talanta NMR Figures',
    'node_description': 'Extracted figures.',
    'node_url': 'www.url.com',
    'submission_date': '2018-5-25',
    'public_release_date': '2006-03-13'},
   [Comment(body='I manually pulled this points out with a web tool.', name='Comment00566')]),
  '3d77bbee-25ec-4358-99a9-636001082efa': ('Potassium Hydroxide Assay',
   [Comment(body='I manually pulled this points out with a web tool.', name='Comment00573')]),
  '63babb6e-a17c-4c60-bcf3-b52a1b717a6f': SampleNode(comments=[], factors=[Factor(csv_column_index=0, decimal_value=0.0, factor_type='Measurement Condition', name='Factor00574', reference_value='', string_value='', unit_reference='Molar')], name='SampleNode00577', sample_name='Potassium Hydroxide', sources=[], species=[SpeciesFactor(name='SpeciesFactor00575', species_reference='K+', stoichiometry=1.0), SpeciesFactor(name='SpeciesFactor00576', species_reference='OH-', stoichiometry=1.0)

---